# Import the libraries

In [39]:
import pandas as pd
from linearmodels import PanelOLS
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Set the font to Times New Roman
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'

# Prepare the data

In [40]:
ind_col = 'naicsh6'
nlp_model = 'mpnet'
doc2vec_vector_size = 768 # 768 for mpnet, 1536 for ada

data = pd.read_pickle(f'../data/all_reg_vars_{nlp_model}_{str(doc2vec_vector_size)}_{ind_col}.pkl')
data.replace([np.inf, -np.inf], np.nan, inplace=True)


### Create lead variables for the DVs

In [41]:
# Sort data by GVKEY and year
reg_data = data.sort_values(by=['GVKEY', 'year'])
# Lead dependent variables by one year
reg_data['numrec_mean_0_lead'] = reg_data.groupby('GVKEY')['numrec_mean_0'].shift(-1)
reg_data['meanrec_mean_descend_lead'] = reg_data.groupby('GVKEY')['meanrec_mean_descend'].shift(-1)
reg_data['year_lead'] = reg_data.groupby('GVKEY')['year'].shift(-1)
reg_data['year_diff'] = reg_data['year_lead'] - reg_data['year']
reg_data = reg_data[reg_data['year_diff'] <=2 ]
reg_data['constant'] = 1

In [42]:
# Limit the data to non-exemplar firms and industries with at least 10 firms
reg_data = reg_data[(reg_data[f'exemplar_{ind_col}'] == 0) & (reg_data[f'no_firms_{ind_col}'] > 4)]

In [43]:
reg_data['year_lead'] = reg_data.groupby('GVKEY')['year'].shift(-1)


### Rename the variables to make them more readable

In [44]:
# define a dictionary that maps old names to new names
rename_dict = {
    'numrec_mean_0_lead': 'Analyst Coverage',
    'meanrec_mean_descend_lead': 'Analyst Recoms',
    f'exemplar_sim_{ind_col}': 'Exemplar Similarity',
    'sale_wins_1': 'Total Sales',
    'n_emp': 'Firm Size',
    'EPS_wins_1': 'EPS',
    'slack_avail_wins_1': 'Available Slack',
    'rd_f_wins_1': 'R&D Expenditure',
    'adv_f_wins_1': 'Advertising Expenditure',
    'dpt_f_wins_1': 'Depreciation Ratio',
    'intang_f_wins_1': 'Intangible Assets Ratio',
    'n_segments': 'No. Segments',
    f'market_share_{ind_col}': 'Market Share',
    'mergers_wins_1': 'Mergers (expenditure)',
    'leverage_wins_1': 'Financial Leverage',
    'is_spx500': 'S&P500 Dummy',
    f'sim_mean_{ind_col}': 'Firm Typicality',
    'strong_weak_modal_ratio': 'Strong-Weak Modals Ratio',
    'positive_negative_ratio': 'Positive-Negative Words Ratio',
    'N_Litigious': 'Litigous Words Ratio',
    f'n_analysts_{ind_col}': 'No. Analysts in Industry',
    f'no_firms_{ind_col}': 'No. Firms in Industry',
    f'category_coherence_{ind_col}': 'Category Coherence',
    f'numrec_avg_year_{ind_col}': 'Average Coverage (Year-Ind)',
    f'meanrec_avg_year_{ind_col}': 'Average Recoms (Year-Ind)',
    f'ind_sim_all_{ind_col}_distinct': 'Category Distinctiveness',
    f'ind_vecs_change_{ind_col}': 'Category Instability',
    f'hhi_{ind_col}': 'Industry HHI',
    f'EPS_wins_1_ex_{ind_col}': 'Exemplar EPS',
    f'exemplar_{ind_col}_typicality': 'Exemplar Typicality',
    'constant': 'Constant'
}

# rename columns
reg_data = reg_data.rename(columns=rename_dict)


### Define the list of independent/control variables


In [45]:
iv = ['Exemplar Similarity']
moderators = ['Category Coherence', 'Category Distinctiveness','Exemplar Typicality']
controls = [
            'Total Sales', 'Firm Size','Market Share', 'EPS', 'Available Slack', 'R&D Expenditure',
            'Advertising Expenditure', 'Intangible Assets Ratio', 'Depreciation Ratio', 
            'Firm Typicality',  'No. Segments',
             'Mergers (expenditure)', 'Financial Leverage', 'S&P500 Dummy', 
             'No. Analysts in Industry', 'No. Firms in Industry',  
             'Average Coverage (Year-Ind)', 'Average Recoms (Year-Ind)',
             'Category Instability', 'Industry HHI', 'Exemplar EPS',
             'Constant']
dv = ['Analyst Coverage', 'Analyst Recoms']

In [46]:
# Set index to GVKEY and year
reg_data = reg_data.set_index(['GVKEY', 'year'])

# Regression Models

## Effect of Exemplar similarity on Analyst Coverage

In [47]:
regressors = iv + moderators + controls
model1 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res1 = model1.fit(cov_type='clustered', cluster_entity=True)
res1


Dep. Variable:,Analyst Coverage,R-squared:,0.1286
Estimator:,PanelOLS,R-squared (Between):,0.2322
No. Observations:,51216,R-squared (Within):,0.1286
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.2040
Time:,16:12:47,Log-likelihood,-1.164e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,253.03
Entities:,8324,P-value,0.0000
Avg Obs:,6.1528,Distribution:,"F(25,42867)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,37.556


## Effect of Exemplar Similarity on Analyst Recommendations

In [48]:
regressors = iv + moderators + controls
model2 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res2 = model2.fit(cov_type='clustered', cluster_entity=True)
res2

Dep. Variable:,Analyst Recoms,R-squared:,0.0554
Estimator:,PanelOLS,R-squared (Between):,0.0351
No. Observations:,33992,R-squared (Within):,0.0554
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0393
Time:,16:12:47,Log-likelihood,-1.411e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,64.256
Entities:,6598,P-value,0.0000
Avg Obs:,5.1519,Distribution:,"F(25,27369)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,32.234


## Moderation Effect of Category Coherence

In [49]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Category Coherence'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Coherence']
regressors = iv + moderators + [interaction_term] + controls

model3 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res3 = model3.fit(cov_type='clustered', cluster_entity=True)
res3


Dep. Variable:,Analyst Coverage,R-squared:,0.1314
Estimator:,PanelOLS,R-squared (Between):,0.2270
No. Observations:,51216,R-squared (Within):,0.1314
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.1985
Time:,16:12:48,Log-likelihood,-1.163e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,249.45
Entities:,8324,P-value,0.0000
Avg Obs:,6.1528,Distribution:,"F(26,42866)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,37.507


In [50]:
interaction_term = 'Exemplar Similarity x Category Coherence'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Coherence']
regressors = iv + moderators + [interaction_term] + controls

model4 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res4 = model4.fit(cov_type='clustered', cluster_entity=True)
res4

Dep. Variable:,Analyst Recoms,R-squared:,0.0557
Estimator:,PanelOLS,R-squared (Between):,0.0380
No. Observations:,33992,R-squared (Within):,0.0557
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0430
Time:,16:12:48,Log-likelihood,-1.411e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,62.037
Entities:,6598,P-value,0.0000
Avg Obs:,5.1519,Distribution:,"F(26,27368)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,31.294


## Moderation Effect of Category Distinctiveness

In [51]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Category Distinctiveness'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Distinctiveness']
regressors = iv + moderators + [interaction_term] + controls

model5 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res5 = model5.fit(cov_type='clustered', cluster_entity=True)
res5


Dep. Variable:,Analyst Coverage,R-squared:,0.1306
Estimator:,PanelOLS,R-squared (Between):,0.2359
No. Observations:,51216,R-squared (Within):,0.1306
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.2068
Time:,16:12:48,Log-likelihood,-1.163e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,247.76
Entities:,8324,P-value,0.0000
Avg Obs:,6.1528,Distribution:,"F(26,42866)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,37.101


In [52]:
interaction_term = 'Exemplar Similarity x Category Distinctiveness'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Distinctiveness']
regressors = iv + moderators + [interaction_term] + controls

model6 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res6 = model6.fit(cov_type='clustered', cluster_entity=True)
res6

Dep. Variable:,Analyst Recoms,R-squared:,0.0557
Estimator:,PanelOLS,R-squared (Between):,0.0266
No. Observations:,33992,R-squared (Within):,0.0557
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0336
Time:,16:12:49,Log-likelihood,-1.411e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,62.113
Entities:,6598,P-value,0.0000
Avg Obs:,5.1519,Distribution:,"F(26,27368)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,31.060


## Moderation Effect of Exemplar Typicality

In [53]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Exemplar Typicality'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Exemplar Typicality']
regressors = iv + moderators + [interaction_term] + controls

model7 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res7 = model7.fit(cov_type='clustered', cluster_entity=True)
res7

Dep. Variable:,Analyst Coverage,R-squared:,0.1297
Estimator:,PanelOLS,R-squared (Between):,0.2314
No. Observations:,51216,R-squared (Within):,0.1297
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.2029
Time:,16:12:49,Log-likelihood,-1.164e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,245.77
Entities:,8324,P-value,0.0000
Avg Obs:,6.1528,Distribution:,"F(26,42866)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,37.024


In [54]:
interaction_term = 'Exemplar Similarity x Exemplar Typicality'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Exemplar Typicality']
regressors = iv + moderators + [interaction_term] + controls

model8 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res8 = model8.fit(cov_type='clustered', cluster_entity=True)
res8

Dep. Variable:,Analyst Recoms,R-squared:,0.0560
Estimator:,PanelOLS,R-squared (Between):,0.0382
No. Observations:,33992,R-squared (Within):,0.0560
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0431
Time:,16:12:50,Log-likelihood,-1.41e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,62.498
Entities:,6598,P-value,0.0000
Avg Obs:,5.1519,Distribution:,"F(26,27368)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,31.380


# Output regression results

In [55]:
def create_table(*results):
    table = pd.DataFrame()
    for i, result in enumerate(results, start=1):
        coefficients = result.params
        pvalues = result.pvalues
        nobs = result.nobs
        rsquared = "{:.3f}".format(result.rsquared)
        nentities = result._entity_info['total']

        # Create a DataFrame for this result
        data = {}
        for key in coefficients.keys():
            data[key] = ["{:.3f}".format(coefficients[key])]
            data[key + ' p-val'] = ["'({:.3f})".format(pvalues[key])]  # added single quote
        data['Observations'] = [nobs]
        data['R-squared'] = [rsquared]
        data['Entities'] = [nentities]
        df = pd.DataFrame(data)

        # Assign model name to the index
        df.index = ['Model' + str(i)]

        # Concatenate to the final table
        table = pd.concat([table, df], axis=0)
    return table

In [56]:

# Now create the table with your results:
table = create_table(res1, res2, res3, res4, res5, res6, res7, res8)


In [57]:
table.T.to_csv(f'../data/tables/main_regression_results_naicsh_4min.csv')